In [5]:
import pandas as pd
import plotly.express as px
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import pytz

df = pd.read_csv("/content/DATASET.zip")

def preprocess_installs(installs):
    return int(installs.replace(",", "").replace("+", "")) if isinstance(installs, str) and installs.replace(",", "").replace("+", "").isdigit() else 0

df["Installs"] = df["Installs"].apply(preprocess_installs)
df_filtered = df[~df["Category"].str.startswith(("A", "C", "G", "S"))]
df_top_categories = df_filtered.groupby("Category")["Installs"].sum().nlargest(5).reset_index()
df_top = df_filtered[df_filtered["Category"].isin(df_top_categories["Category"])].copy()

world_countries = ["United States", "India", "Germany", "Brazil", "Canada", "France", "Australia", "China", "United Kingdom", "Japan"]
df_top["Country"] = [random.choice(world_countries) for _ in range(len(df_top))]

def generate_barchart(df):
    fig_bar = px.bar(df, x="Category", y="Installs", color="Category",
                     title="Top 5 Categories by Installs",
                     text="Installs", color_discrete_sequence=px.colors.qualitative.Set2)
    fig_bar.update_traces(texttemplate='%{text}', textposition='outside')
    fig_bar.update_layout(legend_title_text='Category', legend_orientation="h", legend_y=1.15)
    return fig_bar

def generate_scatterplot(df):
    fig_scatter = px.scatter(df, x="Reviews", y="Installs", color="Category",
                             title="Installs vs. Reviews", size="Installs", hover_name="App")
    fig_scatter.update_layout(legend_title_text='Category', legend_orientation="h", legend_y=-0.2)
    return fig_scatter

ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.datetime.now(ist).time()
allowed_start = datetime.time(18, 0, 0)
allowed_end = datetime.time(20, 0, 0)

if allowed_start <= current_time <= allowed_end:
    df_top_grouped = df_top.groupby(["Country", "Category"])["Installs"].sum().reset_index()

    fig_choropleth = px.choropleth(df_top_grouped, locations="Country", locationmode="country names",
                                   color="Installs", hover_name="Category",
                                   title="Global Installs by Category",
                                   color_continuous_scale="Viridis",
                                   projection="natural earth",
                                   animation_frame="Category")
    fig_choropleth.update_layout(legend_title_text='Installs', legend_orientation="h", legend_y=-0.1)

    fig_bar = generate_barchart(df_top_categories)
    fig_scatter = generate_scatterplot(df_top)

    fig_combined = make_subplots(rows=2, cols=2,
                                 subplot_titles=("Choropleth Map", "Category Bar Chart", "Installs vs Reviews"),
                                 specs=[[{"type": "choropleth"}, {"type": "xy"}],
                                        [{"colspan": 2}, None]])

    fig_combined.add_trace(fig_choropleth.data[0], row=1, col=1)
    for trace in fig_bar.data:
        fig_combined.add_trace(trace, row=1, col=2)
    for trace in fig_scatter.data:
        fig_combined.add_trace(trace, row=2, col=1)

    fig_combined.update_layout(title_text="Interactive Global Installs Dashboard", showlegend=True, legend_orientation="h", legend_y=-0.3)
    fig_combined.show()
else:
    print("This visualization is only available between 6 PM and 8 PM IST.")